### References  

* Turnkey SMS code https://www.twilio.com/docs/guides/how-to-send-sms-messages-in-python
* lat/long conversion to pixels http://stackoverflow.com/questions/1369512/converting-longitude-latitude-to-x-y-on-a-map-with-calibration-points 
x = (total width of image in px) * (180 + latitude) / 360 y = (total height of image in px) * (90 - longitude) / 180
* d3plus.org - API for various chart types
* https://docs.python.org/3/library/json.html - JSON methods
* https://www.thehubway.com/system-data for data
* Dictionary sort using a lambda function - http://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-values-of-the-dictionary-in-python



# Libraries

In [ ]:
import urllib.request, json, time
from twilio.rest import Client 

# Constants

In [ ]:
## The Two Hubway Feed URLs we need
### Name, lat/long,ID
URL_STATION_POSIT = 'https://gbfs.thehubway.com/gbfs/en/station_information.json'

## Online?, capacity, bikes available, last rented, etc
URL_STATION_AVAIL = 'https://gbfs.thehubway.com/gbfs/en/station_status.json'

## When to send out low bike message
ALERT_NUMBER = 5

## Update our feed this often in seconds. 
TTL = 10 

## How long for program to fetch live feed. Reads approximaely 6 times a minute (every 10 seconds)
NUM_LOOPS = 60

# Function Definitions




### I/O Functions (JSON feed input and JSON file output)

In [ ]:
## Standard code for opening URL and returning JSON object
def json_in(url):
    """Given a URL of a JSON feed, returns a dictionary object."""
    
    try:
        response = urllib.request.urlopen(url)
    except urllib.error.HTTPError as e1:
        print(e1)
    else:
        #use json.loads to unpack the json
        return json.loads(response.read().decode('utf-8'))
    

## we'll use a function for less code and cleanliness
def json_out(out_file_name, data, var_text):
    """Takes a JSON formatted dictionary, modifies by adding a Javascript variable assignment, and outputs to 
    a file."""
    with open(out_file_name, mode = 'w') as f:
        f.write("var " + var_text + " =") #Kaleigh added this
        json.dump(data , f,indent = 2)

### Set color status of station. Color levels are arbitrary

In [ ]:

def set_color_status(current_station):
    """Given a quantity of bikes, sets the 'color' of the station to red/yellow/green based upon arbitrary limits."""
    #print(current_station)
    bikes = current_station['num_bikes_available']
    current_station['color'] = 'red' if bikes < 3 else 'yellow' if bikes < 6 else 'green'


### Convert lat/long to x/y pixels
For the visualization software I'm using the points plotted are required to be x,y, not lat/lon.

Challenge was to convert lat/long to x,y screen positions and have the nodes be at the approx relative positions.
Also need to ensure that East/West, North/South is translated correctly to x/y pixel coordinate system. Mostly
trial error, but common sense, like knowing Kenmore should be to the left of South Station on network 
and thus smaller x value helps.

(+71.05)*10000, et al is simply for giving a more granular pixel result, since plotting without this adjustment
gives us pixels VERY close to each other. Basically adjusting to fit and fill the screen.

Reference cited for initial formula, adjustment was mine using trial and error.

In [ ]:
## GOOD CODE -4/26###

## Convert lat/long to x/y screen position. See detailed notes
def lon_to_x(lon):
    """Converts longitude into pixels for display."""
    return round((40)*(90 +(float(lon)+71.05)*10000) / 180,2)

def lat_to_y(lat):
    """Converts latitude into pixels for display"""
    return round((70)*(180 -(float(lat)-42.35)*10000) / 360,2)

### Shorten Name
Some stations have very long names. For consistency and display purposes shorten all names to 5 'words'.

In [ ]:
## shorten the name of station to first 5 words. Here for mainly cleanliness in code below        
def shorten(station):
    """Given a string of words, shortens to 5 words and returns shortened string."""
    return' '.join(station.split( )[:5])

### Use a print function for cell printing. This keeps our code cleaner as we are print various dictionary items and it's quite messy

In [ ]:
def print_status_console(station):
    """Prints status of stations with activity during the update loop."""
    
    print('Time reported -',time.ctime(station['last_reported']))
    print(dict_bikes_master[station['station_id']]['name'])
   
    print('Change of',current_bikes_avail - previous_bikes_avail, 'bikes')
    print(current_bikes_avail, '- currently available')
    print(previous_bikes_avail, '- previously available\n')

### SMS Functions

In [ ]:
## This function actually sends the SMS message, via Twilio. Turnkey code from their website.

## set a default value for map_location so we can use this for variety of texts
def send_sms_alert(message, map_location = ''):
    """Sends a sms message via Twilio using credentials saved as environmental variables. Takes a text string
    as a parameter with 2nd option parameter appended to the first. Both parameters make up body of sms."""

    ## Credentials go below
    ACCOUNT_SID = '' ## Your credientials here
    AUTH_TOKEN = ''
 
    client = Client(ACCOUNT_SID, AUTH_TOKEN) #- Rest Client looks for credentials in environ variables
    client.messages.create(
        to="your text to number here", ## format +16175551212 for example
        from_="your Twilio number here",  ## this is TWILIO account phone number
        body = message + map_location)
    
        
    print('SMS message sent ', message) ## Console feedback

##############################################################

## Forms the message to send to the Twilio function to actually send

## default msg (sent most often) will be low bikes; others added as needed
def send_alert(id, msg = 1):
    """Given a station ID and optional message type, forms the message body and then calls send_sms_alert with
    text of message and Apple Maps URL giving station location. Can be extended for other text alert types."""
    ### form the message, then call the actual function that sends it above. This function should be able to send a
    # variety of message types. After message is formed we'll pass it cleanly to the function above
    
    ## we don't need statement below but keeping as a reminder that this dict is able to be referenced whether we
    ## pass it or not as it's passed by reference not value
    global dict_bikes_master
    
    lat = dict_bikes_master[id]['lat']
    lon = dict_bikes_master[id]['lon']
    stat_name = dict_bikes_master[id]['name']
    
    if msg == 1: ## future expansion for other alert messages. As of now msg will ALWAYS == 1 as it's default
        bikes_available = dict_bikes_master[id]['num_bikes_available']
        message_to_send = stat_name +' is low on bikes. ' + str(bikes_available) + ' available.'
        
    ## URL to display station location and name in IOS or Maps app
    map_location = 'http://maps.apple.com/?ll='+str(lat)+','+str(lon)+'&q='+stat_name.replace(' ', '%20')
        
    ## execute Twilio message
    send_sms_alert(message_to_send, map_location)

## Main Code Starts Here!

The two JSON feeds give us the station name, posit, in one feed and the other feed gives us the number of bikes at each station ID. Fetch both and put all data into one master dictionary.  We'll then use this bike quantity as our 'zero' point to compare against and see the  total rental/return activity at each station. We then re-query the feed and the stations that have a change in quantity have activity, which we will visualize. 

#### So this is JSON object for one station. There is last updated time  for the feed (usually every 10 seconds) as well as the station bike information. Right now all we care about is if station 'is_renting', 'num_bikes_available', and 'last_reported'. The 'last_updated' time is important to ensure we hava a current feed; the 'last_reported' time is used to determine which stations to look at (those reported since our last feed query).

#### The feed we are parsing has over 200 stations.

{"last_updated":1493467716,"TTL":10,"data":{"stations":[{"station_id":"3","num_bikes_available":12,"num_bikes_disabled":0,"num_docks_available":3,"num_docks_disabled":0,
"is_installed":1,"is_renting":1,"is_returning":1,"last_reported":1493461399,"eightd_has_available_keys":false}

#### And this is the JSON object for station location. We only query this once for the program, as all are interested in is the lat/long as well as the actual name

{"last_updated":1493467921,"TTL":10,"data":{"stations":
[{"station_id":"3","name":"Colleges of the Fenway - Fenway at Avenue Louis Pasteur","short_name":"B32006","lat":42.340021,"lon":-71.100812,"region_id":10,"rental_methods":["KEY","CREDITCARD"],"capacity":15,"eightd_has_key_dispenser":true}

In [ ]:
## get bikes available at online stations
json_bikes_avail = json_in(URL_STATION_AVAIL)
json_station_locations = json_in(URL_STATION_POSIT)


dict_bikes_master = {}

## station is open so we can plot it and update the mster dict. 220 is a test station we need to exclude
for station in json_bikes_avail['data']['stations']:    
    if station['is_renting'] == 1 and station['station_id']!= '220': 
        
        ## We're not going to use all of the data from the feed but it's easier to get all the data as opposed 
        ## to just what we need
        dict_bikes_master[station['station_id']] = station
        dict_bikes_master[station['station_id']]['total_activity'] = 0 ## Cumlative rental/return.
        
        ## Station color of our viz indicating current bike quantity
        set_color_status(dict_bikes_master[station['station_id']])

### Start of our master dictionary

The only thing missing is lat/long (position of each station) as well as its proper name. We will get that from the 2nd JSON feed and add. Sample below of what dictionary contains so far.

In [ ]:
print(dict_bikes_master['7'])

## Now we just need the posit and the actual station name

## Now finish master dict with location and name info

In [ ]:


for station in (json_station_locations['data']['stations']):

    ## we just want to add to stations that are open, which is what our master dict contains. json_station_location 
    ## has ALL the stations so we can use try/except to skip the ones which aren't in our master dict, ie. the 
    ## closed ones.
    ## Assignment serves 2nd purpose, i.e. to clean up code below as well
    try:
        dict_cur_stat = dict_bikes_master[station['station_id']] ## temp variable to clean up code
        
        
    except KeyError:
        pass ## i.e. go to the next station in the for loop as this station isn't in our master dict, it's closed
    
    else:
        ## Clean up name for display purposes
        dict_cur_stat['name'] = shorten(station['name'])
        
        ## need lat/lon for sending sms with map posit
        dict_cur_stat['lat'], dict_cur_stat['lon'] = station['lat'], station['lon'] 
                
        ## go ahead and convert to pixels while we are fetching lat and lon
        dict_cur_stat['y'], dict_cur_stat['x'] = lat_to_y(station['lat']), lon_to_x(station['lon'])  
                     
        ## Region is city, i.e. Boston, Cambridge, Somerville, Brookline. Fetch as we might use in the future
        dict_cur_stat['region_id'] = station['region_id']

### Final structure of master dictionary

In [ ]:
print(dict_bikes_master['7'])

So now we have our station posits and baseline activity (0) as well as color status indicating their current quantity. Update the variable for last time compared which is from the feed. The basic infrastructure is in place, now just query the availability feed in semi real time, compare and update our soon to be made JSON output files.

In [ ]:
## update our point of comparison time
## we'll use this later to only look for stations that have 'reported in' ('last_reported') after this time
last_compared_time = json_bikes_avail['last_updated']

# Update Loop Starts Here!

So the only way to determine if a bike has been rented or returned is to compare current availability to previous availability. We have our baseline from above. Now below we will fetch all the bike availability do math, and see changes if any. Then we will update the availability and repeat....


In [ ]:
### MAIN ###########
count = 0


## So that it doesn't run indefinitely, we will loop for specific time, set in the 'Constants' cell. Can run cell
## again without any variables being reset or run a few minutes later. It's just going to compare data to the
## master dict and use the last updated time.

while count < NUM_LOOPS:
    
    count_query_feed = 0 ## if JSON feed has not been updated we requery every 2 seonds for 15 attempts
    
    ## Lists that are written to JSON file
    json_list_bikes_avail = []
    json_list_stations_locations = []
  
    ## pause for the TTL time of the update feed. Don't want to run this right after the above cell has run
    ## so we pause at the beginning of the loop for the TTL time (set in constants). This value is hardcoded
    ## from the Hubway JSON feed and determined by Hubway. We could update less frequently if we are hitting
    ## server too often but this approximates 'real time' data 
    time.sleep(TTL)
    
    
    ## Get latest bike availability
    json_bikes_avail_updated = json_in(URL_STATION_AVAIL)
    
    ## feed is updated every 10 seconds and we are hitting every 10 seconds. In case we are a little early
    ## or feed is a little slow and the updated time hasn't changed, we will pause for 2 seconds and try again
    
    
    while (json_bikes_avail_updated['last_updated'] == last_compared_time) and (count_query_feed < 15):
        if count_query_feed == 14:
            print('\n\nAttempting last update. Please check feed status and try later.\n\n')
        time.sleep(2)
        count_query_feed += 1
        json_bikes_avail_updated = json_in(URL_STATION_AVAIL)
    
    
    ##  display current feed time
    
    ## Convert from epoch time and display current feed time. Ideally should display every 10 seconds
    print('Current feed time is-', time.ctime(json_bikes_avail_updated['last_updated']),'\n\n')
    
#################
    
    ## sort now for display purposes later on screen
    for station in sorted(json_bikes_avail_updated['data']['stations'], \
                          key=lambda k: k['last_reported'], reverse = False):
    

        ## So we're looking for changes in the previous availability at stations. Instead of comparing
        ## current to previous at each station, first see if station has reported in since last update
        ## IF it hasn't then no need to do math and see if number of bikes have changed
        
    
        if (station['last_reported'] > last_compared_time):
            ## we're not going to add new stations that come online after our base case so we
            ## will exclude via try/except. Otherwise if a station comes online after we make our initial
            ## dict then we will be attempting to update a non-existent key.
            
            
            ## only need one statement in the 'try' block. If key error then use except
            ## to pass to the next 'if loop' item, use 'else' statement to execute the rest of statements
            ## below.
            
            ## we need this assigment but are also using it as test to see if we need to ignore a new station
            ## previous_bikes_avail is just temp variable for code cleanliness. its set to what is in 
            ## master dict for current station in jSON feed
            try: 
                
                previous_bikes_avail = dict_bikes_master[station['station_id']]['num_bikes_available']
                
            ## key for current station in JSON feed doesn't exist in our dictionary. We will skip it
            except Exception as e:# KeyError:
#                 print(e)
                pass

            ## no exception so just finish up the loop
            else:
                
                
                ## now looking for stations whose bikes have changed in quantity, 
                ## previous is set from master dict, current is from new JSON feed
                ## previous is temp variable for code cleanliness
                if (previous_bikes_avail != station['num_bikes_available']):
                    current_bikes_avail = station['num_bikes_available']
                
                    ## Print function here
                    ## Print out stations that have changes since last query ~10 seconds prior
                    
                    print_status_console(station)
                    

                    ## update master data since station has changed
                    dict_bikes_master[station['station_id']].update(station)
                    set_color_status(dict_bikes_master[station['station_id']])
                    ## 4/25
                    ## absolute value as we are just interested in total activity for our activity tracker.
                    dict_bikes_master[station['station_id']]['total_activity'] +=\
                    abs(current_bikes_avail - previous_bikes_avail)
                    
                    ## Send text message about low bikes
                    if station['num_bikes_available'] < ALERT_NUMBER:
#                         pass
                        send_alert(station['station_id']) 
                    
            #### AT this point our master dict should be updated with current bike station status
            #### Now just copy stations that have positive cumlative activity to JSON file and write out
            #### Two JSON feeds needed, one with active stations and their posits
            #### Second feed active station and bike activity

    for station in dict_bikes_master:
            if dict_bikes_master[station]['total_activity'] > 0:
                
                ## we start from blank list each loop and append activity. Granted we are probably changing
                ## only a few items and rewriting unchanged ones but since it's a list this is easiest way
                
                json_list_bikes_avail.append({#"ID": dict_bikes_master[station]['station_id'],\
                        "name": dict_bikes_master[station]['name'],\
                        "Bikes Available": dict_bikes_master[station]['num_bikes_available'],\
                        "Updated": time.ctime(dict_bikes_master[station]['last_reported']),\
                        "Total Activity": dict_bikes_master[station]['total_activity'],\
                        "color": dict_bikes_master[station]['color']})
                
                ## Now do the same thing for the location list
                
                json_list_stations_locations.append({"name": dict_bikes_master[station]['name'],\
                        "Last Reported": dict_bikes_master[station]['last_reported'],\
                        #"ID": dict_bikes_master[station]['station_id'],\
                        "x": lon_to_x(dict_bikes_master[station]['lon']),\
                            "y": lat_to_y(dict_bikes_master[station]['lat'])})
            

    ## Order so that newest activity is on top in viz (if overlapping of plotting)
    json_list_stations_locations = sorted(json_list_stations_locations, \
                                      key=lambda k: k['Last Reported'], reverse = False) 
    

    ## write out our JSON files
    json_out('project_data.json', json_list_bikes_avail, 'data')
    json_out('project_posits.json', json_list_stations_locations, 'positions')


            
    ## update our point of comparison time
    last_compared_time = json_bikes_avail_updated['last_updated']
    
    

    
    count +=1

In [ ]:
for station in dict_bikes_master:
    if dict_bikes_master[station]['total_activity'] > 0:
        print(dict_bikes_master[station]['total_activity'], dict_bikes_master[station]['name'])
